# Analyze Wines

In [66]:
# Imports
import pandas as pd
import numpy as np
from datetime import datetime
import requests
import matplotlib.pyplot as plt
import json

### Part 1

In [67]:
regions = pd.read_csv("regions.csv")
print(f"Number of regions(Number of rows): {regions.shape[0]}")

Number of regions(Number of rows): 4353


In [68]:
regions["NomRegion"].values.sort()
regions

,NomRegion
0,Alpes du sud
1,Alpes du sud
2,Alpes du sud
3,Alpes du sud
4,Alsace
...,...
4348,Vins du Monde
4349,Vins du Monde
4350,Vins du Monde
4351,Vins du Monde


In [69]:
# To remove the extra space in the end of the values
regions["NomRegion"] = [" ".join(i.split(" ")[:-1]) if i.endswith(" ") else i for i in regions["NomRegion"]]
# Since "Jura-Savoie" and "Provence-Corse" exist where "-" cannot be replaced with " "
# I do the below specific replaces

replaces = {"Languedoc-Roussillon" : "Languedoc Roussillon",
            "Languedoc" : "Languedoc Roussillon",
            "Sud-Ouest" : "Sud Ouest",
            "Vallée-de-la-Loire" : "Vallée de la Loire",
            "Vallée-du-Rhône" : "Vallée du Rhône",
            "Roussillon" : "Languedoc Roussillon",
            "Jura" : "Jura-Savoie",
            "Savoie" : "Jura-Savoie",
            "Monde" : "Vins du Monde",
            "Provence" : "Provence-Corse",
            "Corse" : "Provence-Corse"}
regions["NomRegion"].replace(replaces, inplace = True)

In [70]:
regions.drop_duplicates(inplace = True)
regions

,NomRegion
0,Alpes du sud
4,Alsace
209,Beaujolais
354,Bordeaux
1375,Bourgogne
1713,Champagne
2065,Provence-Corse
2074,Jura-Savoie
2096,Languedoc Roussillon
2671,Loire


### Part 2

In [71]:
carrefour = pd.read_csv("Carrefour UTF8.csv")
print(f"Names of columns in carrefour dataset = \n{carrefour.columns.values}")

Names of columns in carrefour dataset = 
['Producteur' 'AssemblageCepage' 'Millesime' 'Region' 'Pays' 'Contenance'
 'ContenanceUnit' 'DegreeAlcool' 'appellation' 'CommBouche' 'CommOeil'
 'CommNez' 'CommVin' 'Temperature' 'BestBefore' 'BestAfter' 'Prix'
 'PrixDevise' 'DateCrawl' 'Couleur' 'UrlImage' 'UrlPageVin'
 'UrlSiteSource' 'NomVin']


In [72]:
print(f"Number of Wines = {carrefour.shape[0]}")

Number of Wines = 1000


In [73]:
# Pre Cleaning
carrefour["Region"] = [str(i) for i in carrefour["Region"]]
carrefour["Region"] = [" ".join(i.split(" ")[:-1]) if i.endswith(" ") else i for i in carrefour["Region"]]
carrefour["Region"].replace(replaces, inplace = True)

In [74]:
regions_carrefour = pd.merge(regions, carrefour, how = "inner", left_on="NomRegion", right_on="Region")

In [75]:
# Cleaning the data
# Since I saw all the values in columns of "BestBefore" and "BestAfter" are nan
# I Assume we dont need the "PrixDevise"
regions_carrefour.drop(["PrixDevise", "NomRegion"], inplace=True, axis=1)

In [76]:
# Which varieties of wine are defined ?

# I'm not sure if this is correct
regions_carrefour[regions_carrefour["CommBouche"].notnull()]

,Producteur,AssemblageCepage,Millesime,Region,Pays,Contenance,ContenanceUnit,DegreeAlcool,appellation,CommBouche,...,Temperature,BestBefore,BestAfter,Prix,DateCrawl,Couleur,UrlImage,UrlPageVin,UrlSiteSource,NomVin
0,La Cave d'Augustin Florent,Riesling (100%),2016,Alsace,France,37.5,cl,12.0,Alsace,Une couleur jaune clair aux reflets verts.,...,9.0,NaN,NaN,3.05,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/la-cav...,https://grandsvins.carrefour.fr,"La cave d'Augustin Florent ""riesling"" 2016"
1,La Cave d'Augustin Florent,Sylvaner,2017,Alsace,France,75.0,cl,12.0,Alsace,Une jolie couleur or pâle.,...,9.5,NaN,NaN,3.50,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/la-cav...,https://grandsvins.carrefour.fr,"La Cave d'Augustin Florent ""Sylvaner"" Blanc 2017"
2,Cave de Ribeauvillé,Riesling,2015,Alsace,France,37.5,cl,13.0,Alsace,La robe est d'un jaune brillant aux reflets do...,...,9.0,NaN,NaN,4.30,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/consta...,https://grandsvins.carrefour.fr,"Constance Muller Riesling ""Reflets de France"" ..."
3,Lucien Edelstein,NaN,2016,Alsace,France,75.0,cl,12.0,Alsace,La robe est d'un jaune pâle et léger.,...,9.0,NaN,NaN,4.35,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/lucien...,https://grandsvins.carrefour.fr,"Lucien Edelstein ""Edelzwicker"" 2016"
4,Groupe Bestheim,Riesling,2016,Alsace,France,75.0,cl,0.0,Alsace,La robe est d'un jaune pâle brillant et limpide.,...,9.0,NaN,NaN,4.65,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/maison...,https://grandsvins.carrefour.fr,Maison Heim Riesling 2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,Les Vins de Vienne,Syrah,2015,Vallée du Rhône,France,75.0,cl,13.0,Saint-Joseph,La robe est d'un rouge sombre aux reflets viol...,...,17.0,NaN,NaN,15.95,12/10/2018 00:00,Rouge,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/vins-d...,https://grandsvins.carrefour.fr,"Les Vins de Vienne ""Les Farnauds"" 2015"
991,E. Guigal,"Grenache (70%), Syrah (20%), Mourvèdre (10%)",2013,Vallée du Rhône,France,75.0,cl,14.5,Gigondas,Ce vin possède une belle robe pourpre,...,18.0,NaN,NaN,17.95,12/10/2018 00:00,Rouge,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/maison...,https://grandsvins.carrefour.fr,"Maison Guigal ""Gigondas"" 2013"
992,Maison Guigal,Viognier,2016,Vallée du Rhône,France,75.0,cl,13.0,Saint-Joseph,La robe est d'un jaune pâle aux reflets verts.,...,11.0,NaN,NaN,17.95,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/maison...,https://grandsvins.carrefour.fr,"Maison Guigal ""Saint-Joseph"" Blanc 2016"
993,E. Guigal,NaN,2014,Vallée du Rhône,France,75.0,cl,14.5,Gigondas,La robe est d'un rouge rubis pourpre.,...,17.0,NaN,NaN,17.95,12/10/2018 00:00,Rouge,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/maison...,https://grandsvins.carrefour.fr,"Maison Guigal ""Gigondas"" 2014"


In [77]:
# Which wines might be drink this year? 


In [78]:
# Wines from Burgundy and costing more than €19
regions_carrefour[(regions_carrefour["Region"] == "Bourgogne") & (regions_carrefour["Prix"] > 19)]

,Producteur,AssemblageCepage,Millesime,Region,Pays,Contenance,ContenanceUnit,DegreeAlcool,appellation,CommBouche,...,Temperature,BestBefore,BestAfter,Prix,DateCrawl,Couleur,UrlImage,UrlPageVin,UrlSiteSource,NomVin
430,Domaine et Château,Pinot noir,2015,Bourgogne,France,75.0,cl,0.0,Pernand-Vergelesses,Une belle robe grenat.,...,12.0,NaN,NaN,19.80,12/10/2018 00:00,Rouge,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/75-aoc...,https://grandsvins.carrefour.fr,Domaine Virginie Pillet 2015
431,Maison Louis Latour,Chardonnay (100%),2010,Bourgogne,France,75.0,cl,0.0,Saint-Aubin,Une robe limpide jaune d'or.,...,11.0,NaN,NaN,19.90,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/75-aoc...,https://grandsvins.carrefour.fr,Maison Louis Latour 2010
432,Lycée viticole de Beaune,Pinot noir (100%),2016,Bourgogne,France,75.0,cl,0.0,Beaune 1er Cru,La robe est d'un rouge rubis léger.,...,15.0,NaN,NaN,19.90,12/10/2018 00:00,Rouge,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/domain...,https://grandsvins.carrefour.fr,Domaine du Clos Saint-Philibert Rouge 2016
433,Jean-Marie Guffens,Chardonnay (100%),2014,Bourgogne,France,75.0,cl,13.0,Pouilly-Fuissé,La robe est jaune claire.,...,13.0,NaN,NaN,19.95,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/terres...,https://grandsvins.carrefour.fr,"Maison Verget ""Terres de Pierres"" 2014"
434,Famille Bichot,Chardonnay,2016,Bourgogne,France,75.0,cl,13.0,Montagny 1er Cru,Sa couleur est d'un joli jaune clair aux refle...,...,13.0,NaN,NaN,19.95,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/jean-b...,https://grandsvins.carrefour.fr,"Jean Bouchard ""Montagny 1er Cru"" Tastevinage b..."
435,Jean Bouchard,Pinot noir,2010,Bourgogne,France,75.0,cl,13.0,Savigny-Les-Beaune,Une belle couleur rouge sombre aux reflets rubis.,...,15.5,NaN,NaN,19.95,12/10/2018 00:00,Rouge,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/jean-b...,https://grandsvins.carrefour.fr,"Jean Bouchard ""Savigny-Les-Beaune"" rouge bio 2010"
436,Jean Bouchard,Pinot noir,2015,Bourgogne,France,75.0,cl,13.0,Fixin,Une robe aux nuances de rubis intense.,...,15.0,NaN,NaN,19.95,12/10/2018 00:00,Rouge,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/jean-b...,https://grandsvins.carrefour.fr,"Jean Bouchard ""Fixin"" rouge 2015"


In [79]:
# what are the top three most common regions?
regions_carrefour["Region"].value_counts().head(3)

Bordeaux                253
Languedoc Roussillon    149
Bourgogne               121
Name: Region, dtype: int64

In [80]:
# Which query could be executed in order to build this plot?
# Pie Chart
round(regions_carrefour["Region"].value_counts()/len(regions_carrefour["Region"])*100, 2)

Bordeaux                        25.43
Languedoc Roussillon            14.97
Bourgogne                       12.16
Provence-Corse                  11.96
Vallée du Rhône                 10.85
Val de Loire                     7.54
Sud Ouest                        5.63
Alsace                           4.42
Champagne                        2.51
Beaujolais                       1.91
Jura-Savoie                      1.61
Vins du Monde                    0.50
Vallée de la Loire et Centre     0.30
Vallée de la Loire               0.20
Name: Region, dtype: float64

In [81]:
# Which query could be executed in order to build this plot? 
# Histogram plot
regions_carrefour["Region"].value_counts().sort_index().head(12)

Alsace                           44
Beaujolais                       19
Bordeaux                        253
Bourgogne                       121
Champagne                        25
Jura-Savoie                      16
Languedoc Roussillon            149
Provence-Corse                  119
Sud Ouest                        56
Val de Loire                     75
Vallée de la Loire                2
Vallée de la Loire et Centre      3
Name: Region, dtype: int64

In [82]:
#regions_carrefour["Millesime"] = regions_carrefour["Millesime"].apply(lambda x: datetime.now().year if x==0 else x)

In [83]:
regions_carrefour.loc[regions_carrefour["Millesime"] == 0] = datetime.now().year

In [84]:
for_before_after = ["Bordeaux", "Bourgogne", "Alsace"]
top_3_regions = regions_carrefour[regions_carrefour["Region"].isin(for_before_after)]
top_3_regions

,Producteur,AssemblageCepage,Millesime,Region,Pays,Contenance,ContenanceUnit,DegreeAlcool,appellation,CommBouche,...,Temperature,BestBefore,BestAfter,Prix,DateCrawl,Couleur,UrlImage,UrlPageVin,UrlSiteSource,NomVin
0,La Cave d'Augustin Florent,Riesling (100%),2016,Alsace,France,37.5,cl,12.0,Alsace,Une couleur jaune clair aux reflets verts.,...,9.0,NaN,NaN,3.05,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/la-cav...,https://grandsvins.carrefour.fr,"La cave d'Augustin Florent ""riesling"" 2016"
1,La Cave d'Augustin Florent,Sylvaner,2017,Alsace,France,75.0,cl,12.0,Alsace,Une jolie couleur or pâle.,...,9.5,NaN,NaN,3.50,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/la-cav...,https://grandsvins.carrefour.fr,"La Cave d'Augustin Florent ""Sylvaner"" Blanc 2017"
2,Cave de Ribeauvillé,Riesling,2015,Alsace,France,37.5,cl,13.0,Alsace,La robe est d'un jaune brillant aux reflets do...,...,9.0,NaN,NaN,4.30,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/consta...,https://grandsvins.carrefour.fr,"Constance Muller Riesling ""Reflets de France"" ..."
3,Lucien Edelstein,NaN,2016,Alsace,France,75.0,cl,12.0,Alsace,La robe est d'un jaune pâle et léger.,...,9.0,NaN,NaN,4.35,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/lucien...,https://grandsvins.carrefour.fr,"Lucien Edelstein ""Edelzwicker"" 2016"
4,Groupe Bestheim,Riesling,2016,Alsace,France,75.0,cl,0.0,Alsace,La robe est d'un jaune pâle brillant et limpide.,...,9.0,NaN,NaN,4.65,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/maison...,https://grandsvins.carrefour.fr,Maison Heim Riesling 2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,Lycée viticole de Beaune,Pinot noir (100%),2016,Bourgogne,France,75.0,cl,0.0,Beaune 1er Cru,La robe est d'un rouge rubis léger.,...,15.0,NaN,NaN,19.90,12/10/2018 00:00,Rouge,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/domain...,https://grandsvins.carrefour.fr,Domaine du Clos Saint-Philibert Rouge 2016
433,Jean-Marie Guffens,Chardonnay (100%),2014,Bourgogne,France,75.0,cl,13.0,Pouilly-Fuissé,La robe est jaune claire.,...,13.0,NaN,NaN,19.95,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/terres...,https://grandsvins.carrefour.fr,"Maison Verget ""Terres de Pierres"" 2014"
434,Famille Bichot,Chardonnay,2016,Bourgogne,France,75.0,cl,13.0,Montagny 1er Cru,Sa couleur est d'un joli jaune clair aux refle...,...,13.0,NaN,NaN,19.95,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/jean-b...,https://grandsvins.carrefour.fr,"Jean Bouchard ""Montagny 1er Cru"" Tastevinage b..."
435,Jean Bouchard,Pinot noir,2010,Bourgogne,France,75.0,cl,13.0,Savigny-Les-Beaune,Une belle couleur rouge sombre aux reflets rubis.,...,15.5,NaN,NaN,19.95,12/10/2018 00:00,Rouge,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/jean-b...,https://grandsvins.carrefour.fr,"Jean Bouchard ""Savigny-Les-Beaune"" rouge bio 2010"


In [85]:
top_3_regions["Region"].unique()

array(['Alsace', 'Bordeaux', 'Bourgogne'], dtype=object)

In [86]:
top_3_regions.loc[top_3_regions["Region"] == "Bordeaux", "BestAfter"] = top_3_regions["Millesime"] + 3
top_3_regions.loc[top_3_regions["Region"] == "Bourgogne", "BestAfter"] = top_3_regions["Millesime"] + 5
top_3_regions.loc[top_3_regions["Region"] == "Alsace", "BestAfter"] = top_3_regions["Millesime"] + 7

top_3_regions.loc[top_3_regions["Region"] == "Bordeaux", "BestBefore"] = top_3_regions["Millesime"] + 7
top_3_regions.loc[top_3_regions["Region"] == "Bourgogne", "BestBefore"] = top_3_regions["Millesime"] + 10
top_3_regions.loc[top_3_regions["Region"] == "Alsace", "BestBefore"] = top_3_regions["Millesime"] + 4

top_3_regions

D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,Producteur,AssemblageCepage,Millesime,Region,Pays,Contenance,ContenanceUnit,DegreeAlcool,appellation,CommBouche,...,Temperature,BestBefore,BestAfter,Prix,DateCrawl,Couleur,UrlImage,UrlPageVin,UrlSiteSource,NomVin
0,La Cave d'Augustin Florent,Riesling (100%),2016,Alsace,France,37.5,cl,12.0,Alsace,Une couleur jaune clair aux reflets verts.,...,9.0,2020.0,2023.0,3.05,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/la-cav...,https://grandsvins.carrefour.fr,"La cave d'Augustin Florent ""riesling"" 2016"
1,La Cave d'Augustin Florent,Sylvaner,2017,Alsace,France,75.0,cl,12.0,Alsace,Une jolie couleur or pâle.,...,9.5,2021.0,2024.0,3.50,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/la-cav...,https://grandsvins.carrefour.fr,"La Cave d'Augustin Florent ""Sylvaner"" Blanc 2017"
2,Cave de Ribeauvillé,Riesling,2015,Alsace,France,37.5,cl,13.0,Alsace,La robe est d'un jaune brillant aux reflets do...,...,9.0,2019.0,2022.0,4.30,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/consta...,https://grandsvins.carrefour.fr,"Constance Muller Riesling ""Reflets de France"" ..."
3,Lucien Edelstein,NaN,2016,Alsace,France,75.0,cl,12.0,Alsace,La robe est d'un jaune pâle et léger.,...,9.0,2020.0,2023.0,4.35,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/lucien...,https://grandsvins.carrefour.fr,"Lucien Edelstein ""Edelzwicker"" 2016"
4,Groupe Bestheim,Riesling,2016,Alsace,France,75.0,cl,0.0,Alsace,La robe est d'un jaune pâle brillant et limpide.,...,9.0,2020.0,2023.0,4.65,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/maison...,https://grandsvins.carrefour.fr,Maison Heim Riesling 2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,Lycée viticole de Beaune,Pinot noir (100%),2016,Bourgogne,France,75.0,cl,0.0,Beaune 1er Cru,La robe est d'un rouge rubis léger.,...,15.0,2026.0,2021.0,19.90,12/10/2018 00:00,Rouge,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/domain...,https://grandsvins.carrefour.fr,Domaine du Clos Saint-Philibert Rouge 2016
433,Jean-Marie Guffens,Chardonnay (100%),2014,Bourgogne,France,75.0,cl,13.0,Pouilly-Fuissé,La robe est jaune claire.,...,13.0,2024.0,2019.0,19.95,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/terres...,https://grandsvins.carrefour.fr,"Maison Verget ""Terres de Pierres"" 2014"
434,Famille Bichot,Chardonnay,2016,Bourgogne,France,75.0,cl,13.0,Montagny 1er Cru,Sa couleur est d'un joli jaune clair aux refle...,...,13.0,2026.0,2021.0,19.95,12/10/2018 00:00,Blanc,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/jean-b...,https://grandsvins.carrefour.fr,"Jean Bouchard ""Montagny 1er Cru"" Tastevinage b..."
435,Jean Bouchard,Pinot noir,2010,Bourgogne,France,75.0,cl,13.0,Savigny-Les-Beaune,Une belle couleur rouge sombre aux reflets rubis.,...,15.5,2020.0,2015.0,19.95,12/10/2018 00:00,Rouge,https://res.cloudinary.com/elixup/image/upload...,https://grandsvins.carrefour.fr/product/jean-b...,https://grandsvins.carrefour.fr,"Jean Bouchard ""Savigny-Les-Beaune"" rouge bio 2010"


In [87]:
# Write a function called update_dates( dataframe) which receives a dataframe and udapte 
# “Millesime”, “BestBefore”, “BestAfter” columns 

key = 1
j = requests.get(f"http://micgiwines2019.azurewebsites.net/api/GetBestWines2019?key=Groupe{key}").json()
temp = pd.DataFrame(j)
temp["Something"] = ""

In [88]:
temp.loc[temp["appelation"] == "Bourgogne", "Something"] = int(2)

In [89]:
temp

,appelation,millesime,percent,Something
0,Bourgogne,2016,20,2
1,Corse,2016,20,


In [115]:
key = 2

def update_dates(df):
    df.loc[df["millesime"] == 0] = datetime.now().year
    
    df.loc[df["appelation"] == "Bordeaux", "BestAfter"] = df["millesime"] + 3
    df.loc[df["appelation"] == "Bourgogne", "BestAfter"] = df["millesime"] + 5
    df.loc[df["appelation"] == "Alsace", "BestAfter"] = df["millesime"] + 7

    df.loc[df["appelation"] == "Bordeaux", "BestBefore"] = df["millesime"] + 7
    df.loc[df["appelation"] == "Bourgogne", "BestBefore"] = df["millesime"] + 10
    df.loc[df["appelation"] == "Alsace", "BestBefore"] = df["millesime"] + 4

    df.fillna(0, inplace=True)
    df[["BestBefore", "BestAfter"]] = df[["BestBefore", "BestAfter"]].astype(int)
    return df

j = requests.get(f"http://micgiwines2019.azurewebsites.net/api/GetBestWines2019?key=Groupe{key}").json()
temp = pd.DataFrame(j)

update_dates(temp)

,appelation,millesime,percent,BestAfter,BestBefore
0,Bourgogne,2016,5,2021,2026
1,Corse,2016,5,0,0


### Part 3

In [91]:
site = pd.read_csv("Site 4 11 OK.csv")
site["Site"] = ""

In [92]:
site

,$id,Id,Accords,AgeVignes,Appellation,AppellationLabel,BestAfter,BestBefore,AssemblageCepage,Classement,...,Recompenses,NomRegion,TempService,TypeVin,UrlProduit,UrlSiteSource,TypeVendange,NomVignoble,ViticultureLabel,Site
0,1892,1892,NaN,0,Vin de France,NaN,0,0,"Mansois, merlot",NaN,...,NaN,Sud-Ouest,0.0,NaN,https://www.lesgrappes.com/domaine-laurens-com...,https://www.lesgrappes.com/vins,NaN,Domaine Laurens,Traditionnel - Agriculture Traditionnelle / Co...,
1,1893,1893,NaN,0,IGP - Hautes-Alpes,NaN,0,0,Muscat petits grains,NaN,...,NaN,Alpes du sud,0.0,Pétillant,https://www.lesgrappes.com/domaine-allemand-me...,https://www.lesgrappes.com/vins,NaN,Domaine Allemand,Agriculture raisonnée - Agriculture Raisonné -...,
2,1894,1894,NaN,0,Ventoux,NaN,0,0,Clairette 60% Roussanne 20% Grenache Blanc 15%...,NaN,...,NaN,Vallée du Rhône,8.5,NaN,https://www.lesgrappes.com/chateau-saint-pons-...,https://www.lesgrappes.com/vins,NaN,Château Saint Pons,Agriculture raisonnée - Agriculture Raisonné -...,
3,1895,1895,NaN,0,Costières de Nîmes,NaN,0,0,"Syrah 70%, Carignan 20%, Grenache noir 10%",NaN,...,NaN,Vallée du Rhône,0.0,NaN,https://www.lesgrappes.com/chateau-guiot-nimoi...,https://www.lesgrappes.com/vins,NaN,Château guiot,Non Certifié ou En Conversion,
4,1896,1896,Fruit de mer Viande blanche Volaille,0,Charentais,NaN,0,0,"Merlot, Gamay, Cabernet Franc, Cabernet Sauvig...",NaN,...,NaN,Sud-Ouest,0.0,NaN,https://www.lesgrappes.com/domaine-de-poncerea...,https://www.lesgrappes.com/vins,NaN,Domaine de Poncereau de Haut,Agriculture raisonnée - Agriculture Raisonné -...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,3777,3777,Fromage Poisson Viande blanche,0,Chinon,NaN,0,0,Chenin 100 %,NaN,...,NaN,Loire,13.0,NaN,https://www.lesgrappes.com/domaine-dozon-le-sa...,https://www.lesgrappes.com/vins,NaN,Domaine Dozon,Agriculture raisonnée - Agriculture Raisonné -...,
1886,3778,3778,Foie gras Fromage Gibier Viande rouge,0,Maury,NaN,0,0,"Grenache noir 80%, Syrah 20%",NaN,...,NaN,Roussillon,0.0,NaN,https://www.lesgrappes.com/clos-des-vins-damou...,https://www.lesgrappes.com/vins,NaN,Clos Des Vins d’Amour,Non Certifié ou En Conversion,
1887,3779,3779,NaN,0,Vin de France,NaN,0,0,Grenache Syrah Carignan,NaN,...,NaN,Roussillon,0.0,NaN,https://www.lesgrappes.com/cau-eden-perfide-al...,https://www.lesgrappes.com/vins,NaN,Cau Eden,Non Certifié ou En Conversion,
1888,3780,3780,Apéritif Fromage Fruit de mer Poisson,0,Vin de France,NaN,0,0,100% CHENIN,NaN,...,NaN,Loire,0.0,NaN,https://www.lesgrappes.com/le-clos-des-sables-...,https://www.lesgrappes.com/vins,NaN,Le Clos des Sables,"Bio - Certifié Ecocert, Vin Sans sulfite Ajout...",
